# Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from scripts import fill_tn_with_0, \
    create_periodo_dt, \
    create_date_features, \
    scale_tn, \
    create_lag_features,\
    create_ma_features, \
    create_min_features, \
    create_max_features, \
    set_categorical_features, \
    reduce_mem_usage, \
    create_std_features, \
    create_delta_lag_features, \
    set_ordinal_features, \
    limit_categorical_values


In [3]:
plt.rcParams['figure.figsize'] = (12, 6)
sns.set(style='whitegrid', palette='muted', font_scale=1.1)
sns.set_context("notebook", rc={"figure.figsize": (12, 6)})

# Carga de datasets

In [4]:
path_products = './datasets/tb_productos.txt'
path_stocks = './datasets/tb_stocks.txt'
path_sells = './datasets/sell-in.txt'
path_products_to_predict = './datasets/product_id_apredecir201912.txt'

full_dataset_path = './datasets/full_dataset.parquet'
transformed_dataset_path = './datasets/transformed_dataset.parquet'

train_test_path = './datasets/train_test.parquet'

In [5]:
tn_scaler_path='./scalers/scalers.pkl'
encoders_path='./encoders/encoders.pkl'
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size']

In [6]:
#path_products = 'https://drive.usercontent.google.com/u/0/uc?id=1s-AqqbU8AkCquQSTabKQcSyTWJxfzWdC&export=download'
#path_stocks = 'https://drive.usercontent.google.com/u/0/uc?id=1hWem2eGQuxKvRLYowsEr7kEeBLqJMh9y&export=download'
#path_sells = 'https://drive.usercontent.google.com/download?id=1a51g5fWTolz8ffJzqOraoFBbpMrDxDSo&export=download&authuser=0&confirm=t&uuid=7b59490c-94b4-4e15-9493-2679ad9211b6&at=ALoNOgm-5Vr0oaK-fRAcN97ldtSW%3A1748109329167'

In [7]:
df_products = pd.read_csv(path_products, sep='\t', dtype={ 'product_id': str, 'sku_size': str })
df_stocks = pd.read_csv(path_stocks, sep='\t', dtype={ 'product_id': str, 'periodo': str })
df_sell_in = pd.read_csv(path_sells, sep='\t', dtype={ 'product_id': str, 'customer_id': str, 'periodo': str })
df_products_to_predict = pd.read_csv(path_products_to_predict, dtype={ 'product_id': str })

# Feature Engineering

## Lectura de datasets

In [ ]:
df_products

In [ ]:
df_stocks

In [ ]:
df_sell_in

In [ ]:
df_products_to_predict

In [ ]:
df_sell_in[df_sell_in['customer_id'] == '10309']

In [ ]:
df_sell_in[df_sell_in['product_id'] == '20524']

## Unificación del dataset

Un producto empieza a existir en el periodo que tiene la primera venta para algun cliente deja de existir en el periodo que tiene la ultima venta para algun cliente

In [14]:
df_sell_in['periodo'] = pd.to_datetime(df_sell_in['periodo'].astype(str), format='%Y%m')

In [ ]:
df_sell_in[(df_sell_in['customer_id'] == '10312') & (df_sell_in['product_id'] == '20524')]

In [ ]:
product_life = df_sell_in.groupby('product_id')['periodo'].agg(['min', 'max']).rename(columns={
    'min': 'start_p',
    'max': 'end_p'
})

product_life

In [ ]:
product_life.loc['20524']

In [ ]:
customer_life = df_sell_in.groupby('customer_id')['periodo'].agg(['min', 'max']).rename(columns={
    'min': 'start_c',
    'max': 'end_c'
})

customer_life

In [ ]:
customer_life.loc['10309']

In [ ]:
all_periods = pd.date_range(df_sell_in['periodo'].min(), df_sell_in['periodo'].max(), freq='MS')
all_periods

In [ ]:
len(all_periods) * len(df_sell_in['customer_id'].unique()) * len(df_sell_in['product_id'].unique())

In [ ]:
clientes = df_sell_in['customer_id'].unique()
productos = df_sell_in['product_id'].unique()

index = pd.MultiIndex.from_product(
    [clientes, productos, all_periods],
    names=['customer_id', 'product_id', 'periodo']
)

all_combinations = index.to_frame(index=False)
all_combinations

In [ ]:
df_all_combinations = all_combinations.merge(product_life, on='product_id').merge(customer_life, on='customer_id')
df_all_combinations

In [ ]:
#df_complete = df_all_combinations[
#    (df_all_combinations['periodo'] >= df_all_combinations[['start_p', 'start_c']].min(axis=1)) & 
#    (df_all_combinations['periodo'] <= df_all_combinations[['end_p', 'end_c']].max(axis=1))
#]

df_complete = df_all_combinations[
    (df_all_combinations['periodo'] >= df_all_combinations['start_p']) & 
    (df_all_combinations['periodo'] <= df_all_combinations['end_p']) &
    (df_all_combinations['periodo'] >= df_all_combinations['start_c']) &
    (df_all_combinations['periodo'] <= df_all_combinations['end_c'])
]

df_complete = df_complete.drop(columns=['start_p', 'end_p', 'start_c', 'end_c'])

df_complete

In [ ]:
df_sell_in_complete = pd.merge(
    df_complete,
    df_sell_in,
    on=['customer_id', 'product_id', 'periodo'],
    how='left'
)

df_sell_in_complete

In [ ]:
df_sell_in_complete['periodo'] = df_sell_in_complete['periodo'].dt.strftime('%Y%m')
df_sell_in_complete

In [ ]:
df = df_sell_in_complete.merge(df_products, on='product_id').merge(df_stocks, on=['periodo', 'product_id'], how='left')
df

In [28]:
df.drop(columns=['plan_precios_cuidados', 'stock_final', 'cust_request_qty'], inplace=True)

In [ ]:
df.drop(columns=['cust_request_tn'], inplace=True)
df

In [ ]:
last_period = df['periodo'].max()
year = int(last_period[:4])
month = int(last_period[4:])

next_periods = []
for i in range(1,3):
    if month + i > 12:
        next_year = year + 1
        next_month = (month + i) % 12
        if next_month == 0:
            next_month = 12
    else:
        next_year = year
        next_month = month + i
    next_periods.append(f"{next_year}{next_month:02d}")

next_periods

In [ ]:
future_records = []
for period in next_periods:
    temp_df = df[df['periodo'] == last_period].copy()
    temp_df['periodo'] = period
    temp_df['tn'] = None
    future_records.append(temp_df)

df = pd.concat([df] + future_records, ignore_index=True)
df

### Prophet (Pendiente)

## Creo features

In [32]:
df = create_periodo_dt(df)

In [33]:
df = create_date_features(df)

In [34]:
df = fill_tn_with_0(df)

In [35]:
prediction_periods = ['202001', '202002']
test_periods = ['201912']

df_train_test = df[~df['periodo'].isin(prediction_periods)]
df_train = df_train_test[~df_train_test['periodo'].isin(test_periods)]

In [ ]:
scale_tn(df_train, tn_scaler_path, is_train=True)

In [ ]:
df = scale_tn(df, tn_scaler_path, is_train=False)

In [38]:
df = df.sort_values(['periodo_dt'])

In [39]:
df = set_categorical_features(df)

In [40]:
delta_lag_features = create_delta_lag_features(df[['customer_id', 'product_id', 'tn']])

In [41]:
for col in delta_lag_features.columns:
    df[col] = delta_lag_features[col]

In [42]:
del delta_lag_features

In [43]:
lag_features = create_lag_features(df[['customer_id', 'product_id', 'tn']])

In [44]:
for col in lag_features.columns:
    df[col] = lag_features[col]

In [45]:
del lag_features

In [46]:
ma_features = create_ma_features(df[['customer_id', 'product_id', 'tn']])

In [ ]:
for col in ma_features.columns:
    df[col] = ma_features[col]

In [48]:
del ma_features

In [49]:
std_features = create_std_features(df[['customer_id', 'product_id', 'tn']])

In [ ]:
for col in std_features.columns:
    df[col] = std_features[col]

In [51]:
del std_features

In [ ]:
df['mes_conflictivo'] = df['periodo'] == '201908'

In [53]:
df['customer_id'] = df['customer_id'].astype('category')
df['product_id'] = df['product_id'].astype('category')

In [54]:
min_features = create_min_features(df[['customer_id', 'product_id', 'tn']])

In [ ]:
for col in min_features.columns:
    df[col] = min_features[col]

In [56]:
del min_features

In [57]:
max_features = create_max_features(df[['customer_id', 'product_id', 'tn']])

In [ ]:
for col in max_features.columns:
    df[col] = max_features[col]

In [59]:
del max_features

In [60]:
df['customer_id'] = df['customer_id'].astype(str)
df['product_id'] = df['product_id'].astype(str)

In [ ]:
df = limit_categorical_values(df,  columns=['customer_id', 'product_id'])

In [62]:
df['customer_id_limited'] = df['customer_id_limited'].astype(str)
df['product_id_limited'] = df['product_id_limited'].astype(str)

In [ ]:
df = set_ordinal_features(df, ['customer_id_limited', 'product_id_limited'], is_train=True)

In [64]:
df['customer_id'] = df['customer_id'].astype('category')
df['product_id'] = df['product_id'].astype('category')

In [65]:
df = reduce_mem_usage(df)

In [66]:
df['customer_id_limited_encoded'] = df['customer_id_limited_encoded'].astype('category')
df['product_id_limited_encoded'] = df['product_id_limited_encoded'].astype('category')

In [67]:
df.to_parquet(full_dataset_path, index=False)

# Creo datasets de train-test-future

In [68]:
df = pd.read_parquet(full_dataset_path)

### Clusters (Pendiente)